In [1]:
import pandas as pd
import numpy as np

In [13]:
pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import xlrd

In [6]:
dict_df = pd.read_excel('C:/Users/shant/GNIZE/Study/Integrations_Model_Data.xls',
                   sheet_name=['HubSpot Customer Base Metadata','HubSpot Customer Base Value','Integration Ecosystem'])

In [13]:
dict_df['Integration Ecosystem'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32746 entries, 0 to 32745
Data columns (total 9 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Integration ID                            32746 non-null  int64 
 1   Report Month                              32746 non-null  object
 2   Integration Connect Partner? (Yes/No)     32746 non-null  object
 3   Paid Portal Installed Count               32746 non-null  int64 
 4   Free Portal Installed Count               32746 non-null  int64 
 5   Total Installed Count                     32746 non-null  int64 
 6   Integration Total MRR Beginning of Month  32746 non-null  int64 
 7   Integration Total MRR End of Month        32746 non-null  int64 
 8   Integration Total Cancellation Dollars    32746 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 2.2+ MB
